In [16]:
!pip install spacy==3.0.0
!pip install scikit-learn==0.23.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
# import main libraries
from IPython.display import Image
from IPython.core.display import display, HTML
import pandas as pd
from collections import Counter
import random
from spacy import displacy
import json
import os
import spacy
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn


In [18]:
with open('/restauranttrain.bio.txt', 'r') as f:
    for line in f.readlines()[:30]:
        print(line)

B-Rating	2

I-Rating	start

O	restaurants

O	with

B-Amenity	inside

I-Amenity	dining



O	34



B-Rating	5

I-Rating	star

O	resturants

B-Location	in

I-Location	my

I-Location	town



O	98

B-Restaurant_Name	hong

I-Restaurant_Name	kong

O	restaurant

B-Price	reasonable

O	prices



O	a

O	great

O	lunch

O	spot

O	but

B-Hours	open

I-Hours	till



In [19]:
# switch column places
with open('../restauranttrain.bio.txt', 'r') as f:
    with open('restauranttrain_updated.bio', 'w') as w:
        for line in f.readlines():
            if line == '\n':
                w.write(line)
            else:
                w.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [20]:
# split into train and validation
count = 0
with open('../restauranttrain.bio.txt', 'r') as f:
    with open('restauranttrain_updated_train.bio', 'w') as w1:
        with open('restauranttrain_updated_valid.bio', 'w') as w2:
            for line in f.readlines():
                if count < 6500:
                    if line == '\n':
                        w1.write(line)
                        count += 1
                    else:
                        w1.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')
                else:
                    if line == '\n':
                        w2.write(line)
                    else:
                        w2.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [21]:
data = []
labels = []
with open('restauranttrain_updated.bio', 'r') as f:
    cur_data = []
    cur_label = []
    for line in f.readlines():
        if line == '\n':
            data.append(cur_data)
            labels.append(cur_label)
            cur_data = []
            cur_label = []
        else:
            cur_data.append(line.strip().split('\t')[0])
            cur_label.append(line.strip().split('\t')[1])

In [22]:
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

2	B-Rating
start	I-Rating
restaurants	O
with	O
inside	B-Amenity
dining	I-Amenity


In [23]:
!python -m spacy init config base_config.cfg -p ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [24]:
!python -m spacy init fill-config base_config.cfg config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
!python -m spacy convert restauranttrain_updated.bio . -t json -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (1 documents): restauranttrain_updated.json


In [26]:
!python -m spacy convert restauranttrain_updated_train.bio . -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (6500 documents):
restauranttrain_updated_train.spacy


In [27]:
!python -m spacy convert restauranttrain_updated_valid.bio . -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (1160 documents):
restauranttrain_updated_valid.spacy


In [28]:
!python -m spacy train config.cfg --output ./output --paths.train restauranttrain_updated_train.spacy --paths.dev restauranttrain_updated_valid.spacy

ℹ Using CPU

=========================== Initializing pipeline ===========================
Set up nlp object from config
Pipeline: ['tok2vec', 'ner']
Created vocabulary
Finished initializing nlp object
Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.44    0.00    0.00    0.00    0.00
  0     200        248.34   5900.73   55.42   61.62   50.36    0.55
  0     400        480.36   4572.41   68.74   71.09   66.54    0.69
  1     600        645.47   4526.65   73.19   74.67   71.77    0.73
  1     800        773.68   4914.92   77.26   78.39   76.16    0.77
  2    1000        948.89   5124.43   77.36   78.82   75.95    0.77
  3    1200       1141.13   5616.28   78.

In [29]:
!python -m spacy evaluate output/model-last restauranttrain_updated_valid.spacy

ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   80.30
NER R   77.04
NER F   78.63
SPEED   21052


=============================== NER (per type) ===============================

                      P       R       F
Rating            74.54   74.19   74.36
Dish              74.42   76.39   75.39
Cuisine           82.38   80.64   81.50
Location          87.93   83.05   85.42
Hours             81.82   66.00   73.06
Price             91.13   85.61   88.28
Amenity           69.96   63.67   66.67
Restaurant_Name   76.62   77.00   76.81



In [30]:
nlp = spacy.load("output/model-best")
for i in range(10, 20):
    doc = nlp(' '.join(data[i]))

    spacy.displacy.render(doc, style="ent", jupyter=True)